In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.model_selection import train_test_split
from flaml import AutoML
from sklearn.metrics import mean_absolute_error

In [13]:
folder_path = '../データ/train/'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

train = pd.DataFrame()
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    train = pd.concat([train, df], ignore_index=True)
    
test = pd.read_csv('../データ/test.csv')
sample = pd.read_csv('../データ/sample_submission.csv')

C:\Users\81908\AppData\Local\Temp\ipykernel_13280\1450229328.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\81908\AppData\Local\Temp\ipykernel_13280\1450229328.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\81908\AppData\Local\Temp\ipykernel_13280\1450229328.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\81908\AppData\Local\Temp\ipykernel_13280\1450229328.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [14]:
df = pd.concat([train, test], axis=0)
df_ = df.copy()
df_ = df_.set_index('ID')

df_['市区町村コード'] = df_['市区町村コード'].astype(str)
df_['都道府県コード'] = df_['市区町村コード'].astype(str).apply(lambda x: x[:2] if len(x) == 5 else x[:1])
df_['自治体コード'] = df_['市区町村コード'].str[-2:]
# df_['市区町村コード'] = df_['市区町村コード'].astype(int)
# df_['都道府県コード'] = df_['都道府県コード'].astype(int)
# df_['自治体コード'] = df_['自治体コード'].astype(int)

df_['時代'] = df_['建築年'].str.extract(r'([^\d]+)').astype(str)
df_['年'] = df_['建築年'].str.extract(r'(\d+)')
df_['年'] = df_['年'].replace('年','')
df_['年'] = df_['年'].fillna(3000)
df_['年'] = df_['年'].astype(int)
df_['時代'] = df_['時代'].replace({'昭和':1925,'平成':1988,'令和':2018,'nan':3000,'戦前':1945})
df_['西暦'] = df_['時代']+df_['年']
df_['西暦'] = df_['西暦'].apply(lambda x: np.nan if x >= 3000 else x)

df_['取引年'] = df_['取引時点'].str.extract(r'(\d{4})年')
df_['取引年'] = df_['取引年'].astype(int)
df_['四半期'] = df_['取引時点'].str.extract(r'第(\d)四半期')
df_['四半期'] = df_['四半期'].astype(int)

df_['改装'] = df_['改装'].replace({'未改装':0,'改装済':1})

df_['最寄駅：距離（分）'] = df_['最寄駅：距離（分）'].replace({
    '1H?1H30': 60,
    '30分?60分': 30,
    '2H?': 120,
    '1H30?2H': 90,
    })
df_['最寄駅：距離（分）'] = df_['最寄駅：距離（分）'].astype(float)

df_['面積（㎡）'] = df_['面積（㎡）'].replace({'2000㎡以上': 2000})
df_['面積（㎡）'] = df_['面積（㎡）'].astype(int)

df_['取引の事情等'] = df_['取引の事情等'].notnull().astype(int)

df_['建物の構造'] = df_['建物の構造'].fillna('0')
df_['構造_SRC'] = df_['建物の構造'].apply(lambda x: 1 if 'SRC' in x else 0)
df_['構造_RC'] = df_['建物の構造'].apply(lambda x: 1 if 'RC' in x else 0)
df_['構造_鉄骨造'] = df_['建物の構造'].apply(lambda x: 1 if '鉄骨造' in x else 0)
df_['構造_ブロック造'] = df_['建物の構造'].apply(lambda x: 1 if 'ブロック造' in x else 0)
df_['構造_木造'] = df_['建物の構造'].apply(lambda x: 1 if '木造' in x else 0)
df_['構造_軽量鉄骨造'] = df_['建物の構造'].apply(lambda x: 1 if '軽量鉄骨造' in x else 0)

df_['今後の利用目的'] = df_['今後の利用目的'].fillna('0')
df_['今後の利用目的_住宅'] = df_['今後の利用目的'].apply(lambda x: 1 if '住宅' in x else 0)
df_['今後の利用目的_事務所'] = df_['今後の利用目的'].apply(lambda x: 1 if '事務所' in x else 0)
df_['今後の利用目的_店舗'] = df_['今後の利用目的'].apply(lambda x: 1 if '店舗' in x else 0)
df_['今後の利用目的_その他'] = df_['今後の利用目的'].apply(lambda x: 1 if 'その他' in x else 0)

# df_['用途'] = df_['用途'].fillna('0')
# df_['用途_住宅'] = df_['用途'].apply(lambda x: 1 if '住宅' in x else 0)
# df_['用途_店舗'] = df_['用途'].apply(lambda x: 1 if '店舗' in x else 0)
# df_['用途_倉庫'] = df_['用途'].apply(lambda x: 1 if '倉庫' in x else 0)
# df_['用途_工場'] = df_['用途'].apply(lambda x: 1 if '工場' in x else 0)
# df_['用途_駐車場'] = df_['用途'].apply(lambda x: 1 if '駐車場' in x else 0)
# df_['用途_事務所'] = df_['用途'].apply(lambda x: 1 if '事務所' in x else 0)
# df_['用途_作業場'] = df_['用途'].apply(lambda x: 1 if '作業場' in x else 0)
# df_['用途_その他'] = df_['用途'].apply(lambda x: 1 if 'その他' in x else 0)

# label_encoder = LabelEncoder()
# df_['都市計画_encoded'] = label_encoder.fit_transform(df_['都市計画'])
df_['都市計画'] = df_['都市計画'].fillna('0')
df_['都市計画_商業地域'] = df_['都市計画'].apply(lambda x: 1 if '商業地域' in x else 0)
df_['都市計画_第２種中高層住居専用地域'] = df_['都市計画'].apply(lambda x: 1 if '第２種中高層住居専用地域' in x else 0)
df_['都市計画_第１種住居地域'] = df_['都市計画'].apply(lambda x: 1 if '第１種住居地域' in x else 0)
df_['都市計画_準工業地域'] = df_['都市計画'].apply(lambda x: 1 if '準工業地域' in x else 0)
df_['都市計画_近隣商業地域'] = df_['都市計画'].apply(lambda x: 1 if '近隣商業地域' in x else 0)
df_['都市計画_第１種中高層住居専用地域'] = df_['都市計画'].apply(lambda x: 1 if '第１種中高層住居専用地域' in x else 0)
df_['都市計画_第２種住居地域'] = df_['都市計画'].apply(lambda x: 1 if '第２種住居地域' in x else 0)
df_['都市計画_工業地域'] = df_['都市計画'].apply(lambda x: 1 if '工業地域' in x else 0)
df_['都市計画_準住居地域'] = df_['都市計画'].apply(lambda x: 1 if '準住居地域' in x else 0)
df_['都市計画_第１種低層住居専用地域'] = df_['都市計画'].apply(lambda x: 1 if '第１種低層住居専用地域' in x else 0)
df_['都市計画_工業専用地域'] = df_['都市計画'].apply(lambda x: 1 if '工業専用地域' in x else 0)
df_['都市計画_第２種低層住居専用地域'] = df_['都市計画'].apply(lambda x: 1 if '第２種低層住居専用地域' in x else 0)
df_['都市計画_都市計画区域外'] = df_['都市計画'].apply(lambda x: 1 if '都市計画区域外' in x else 0)
df_['都市計画_市街地区域及び市街化調整区域外の都市計画区域'] = df_['都市計画'].apply(lambda x: 1 if '市街地区域及び市街化調整区域外の都市計画区域' in x else 0)
df_['都市計画_市街化調整区域'] = df_['都市計画'].apply(lambda x: 1 if '市街化調整区域' in x else 0)
df_['都市計画_準都市計画区域'] = df_['都市計画'].apply(lambda x: 1 if '準都市計画区域' in x else 0)

df_ = df_.drop({
    '種類',
    '地域',
    '土地の形状',
    '間口',
    '延床面積（㎡）',
    '前面道路：方位',
    '前面道路：種類',
    '前面道路：幅員（ｍ）',
    '時代',
    '年',
    '建築年',
    '都道府県名',
    '市区町村名',
    '地区名',
    '取引時点',
    # '最寄駅：名称',
    '建物の構造',
    # '用途',
    '都市計画',
    '今後の利用目的',
    # '市区町村コード',
    }, axis=1)

df_.head()

,市区町村コード,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,用途,建ぺい率（％）,容積率（％）,改装,取引の事情等,...,都市計画_第２種住居地域,都市計画_工業地域,都市計画_準住居地域,都市計画_第１種低層住居専用地域,都市計画_工業専用地域,都市計画_第２種低層住居専用地域,都市計画_都市計画区域外,都市計画_市街地区域及び市街化調整区域外の都市計画区域,都市計画_市街化調整区域,都市計画_準都市計画区域
ID,,,,,,,,,,,,,,,,,,,,,
1000248,1101,西１８丁目,1.0,１ＤＫ,25,住宅,80.0,400.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1052783,1105,南平岸,14.0,２ＬＤＫ＋Ｓ,50,住宅,NaN,NaN,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1026319,1103,札幌(ＪＲ),8.0,１Ｋ,15,住宅,80.0,400.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1095145,1110,福住,60.0,４ＬＤＫ,80,住宅,60.0,200.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
1093724,1202,中央病院前,2.0,２ＬＤＫ,70,住宅,80.0,400.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 838755 entries, 1000248 to 47006980
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   市区町村コード                      838755 non-null  object 
 1   最寄駅：名称                       835897 non-null  object 
 2   最寄駅：距離（分）                    815471 non-null  float64
 3   間取り                          805723 non-null  object 
 4   面積（㎡）                        838755 non-null  int32  
 5   用途                           738736 non-null  object 
 6   建ぺい率（％）                      812216 non-null  float64
 7   容積率（％）                       812216 non-null  float64
 8   改装                           752581 non-null  float64
 9   取引の事情等                       838755 non-null  int32  
 10  取引価格（総額）_log                 820017 non-null  float64
 11  都道府県コード                      838755 non-null  object 
 12  自治体コード                       838755 non-null  object 
 

In [16]:
df_.isnull().sum()

市区町村コード                             0
最寄駅：名称                           2858
最寄駅：距離（分）                       23284
間取り                             33032
面積（㎡）                               0
用途                             100019
建ぺい率（％）                         26539
容積率（％）                          26539
改装                              86174
取引の事情等                              0
取引価格（総額）_log                    18738
都道府県コード                             0
自治体コード                              0
西暦                              25174
取引年                                 0
四半期                                 0
構造_SRC                              0
構造_RC                               0
構造_鉄骨造                              0
構造_ブロック造                            0
構造_木造                               0
構造_軽量鉄骨造                            0
今後の利用目的_住宅                          0
今後の利用目的_事務所                         0
今後の利用目的_店舗                          0
今後の利用目的_その他                         0
都市計画_商業地域   

In [17]:
df_train_ = df_.dropna(subset=['取引価格（総額）_log'])
df_test_ = df_[df_['取引価格（総額）_log'].isnull()]
df_test_ = df_test_.drop('取引価格（総額）_log', axis=1)

In [18]:
x = df_train_.drop('取引価格（総額）_log', axis=1)
t = df_train_['取引価格（総額）_log']
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.2, random_state=42)

automl = AutoML()
automl_settings = {
    "time_budget": 60,  # 最大実行時間（秒）
    "metric": 'mae',  # 評価指標
    "task": 'regression',  # タスク（classification or regression）
    # "seed": 42,
    "seed": 15,
    # "log_file_name": 'automl.log',  # ログファイルの名前
}
automl.fit(X_train=x_train, y_train=t_train, **automl_settings)

# テストデータでの性能評価
prepre = automl.predict(x_train)
pre = automl.predict(x_test)

[flaml.automl.logger: 05-08 10:57:08] {1680} INFO - task = regression
[flaml.automl.logger: 05-08 10:57:08] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 05-08 10:57:09] {1789} INFO - Minimizing error metric: mae
[flaml.automl.logger: 05-08 10:57:09] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 05-08 10:57:09] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-08 10:57:09] {2345} INFO - Estimated sufficient time budget=92549s. Estimated necessary time budget=791s.
[flaml.automl.logger: 05-08 10:57:09] {2392} INFO -  at 10.3s,	estimator lgbm's best error=0.2309,	best estimator lgbm's best error=0.2309
[flaml.automl.logger: 05-08 10:57:09] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-08 10:57:09] {2392} INFO -  at 10.5s,	estimator lgbm's best error=0.2224,	best estimator lgbm's best error=0.2224
[flaml.automl.logger: 05-08 10:5

In [19]:
print(mean_absolute_error(t_train, prepre))
print(mean_absolute_error(t_test, pre))

0.0735694969320653
0.0786916841168959


In [20]:
prediction = automl.predict(df_test_)
prediction

array([7.32822794, 6.6969684 , 6.47338201, ..., 7.37411443, 7.49653281,
       6.94033797])

In [21]:
sample['取引価格（総額）_log'] = prediction
sample.head()

,ID,取引価格（総額）_log
0,1000107,7.328228
1,1000109,6.696968
2,1000110,6.473382
3,1000114,6.687333
4,1000115,6.530693


In [22]:
# sample.to_csv('../提出ファイル/submission_auto_11.csv', index=False, header=True)